<a href="https://colab.research.google.com/github/jodog0412/US-STOCK-PREDICTION/blob/main/US_STOCK_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare

In [ ]:
!pip install yfinance
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

In [ ]:
from pandas.compat.pyarrow import pa
import sklearn
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as prep
from sklearn.metrics import mean_squared_error

ticker_ml=yf.Ticker("MSFT")
data_raw=ticker_ml.history(period="5y")

X=data_raw.drop(['Close'],axis=1)
Y=data_raw['Close']
plt.plot(data_raw.index,Y)

# Data Scailing

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,
                                               Y,
                                               test_size=0.2)
scale = prep.StandardScaler()
x_train = scale.fit_transform(x_train)
x_test = scale.transform(x_test)


# MLP

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1=torch.nn.Sequential(
            nn.Linear(6,16),
            nn.ReLU()
        )
        self.layer2=torch.nn.Sequential(
            nn.Linear(16,32),
            nn.ReLU()
        )
        self.layer3=torch.nn.Sequential(
            nn.Linear(32,64),
            nn.ReLU()
        )
        self.out=torch.nn.Sequential(
            nn.Linear(64,1)
        )
    def forward(self,x):
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        output=self.out(x)
        return output 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
model=MLP().to(device)
criterion=nn.MSELoss().to(device)
optimizer=optim.Adam(model.parameters(),lr=0.001)
x_train_T=torch.Tensor(x_train.astype(np.float64))
y_train_T=torch.Tensor(y_train.astype(np.float64))
losses=[]
epochs=10000
def train(model,optimizer):
    model.train()
    for epoch in range(epochs):
        running_loss=0.0
        pred=model(x_train_T.cuda())
        loss=criterion(pred.cuda(),y_train_T.reshape(-1,1).cuda())
        optimizer.zero_grad() #배치마다 optimizer 초기화
        loss.backward() #손실함수 기준 역전파 
        optimizer.step() #가중치 최적화
        running_loss += loss.item()
        losses.append(running_loss)
        if epoch%100==0:
            print('[%d] Train loss: %.5f' %(epoch, running_loss))
    plt.plot(list(range(epochs)),losses)
train(model,optimizer)

In [ ]:
y_pred=model(x_train_T.cuda())
stock_pred=pd.DataFrame({"Pred":y_pred.flatten().detach().cpu().numpy()},index=pd.date_range(start=data_raw.index[-1], 
                                                                                   periods=len(y_pred), freq="M"))
plt.plot(Y[-730:])
plt.plot(stock_pred[:12])